# Run Streamlit and Bedrock on SageMaker notebook instance

This notebook was built using the conda PyTorch 3.10 kernel on an `ml.t3.medium` instance.

In [ ]:
!pip install streamlit

In [ ]:
%%writefile language_app.py

import streamlit as st

def add_to_session_history(role, content):
    st.session_state.messages.append({"role": role, "content": content})
    

# this adds a title to the application
st.title('My first language application')

# create a local list to hold the mesage history
st.session_state.messages = []


# this prints all the messages in the history
for msg in st.session_state.messages:
    with st.chat_message(msg['role']):
        st.markdown(msg['content'])

# this creates a template on the screen and a variable in the script
prompt = st.chat_input('This is where you put your questions')

if prompt:
    
    st.chat_message('user').markdown(prompt)

    
    add_to_session_history('user', prompt)
    
    res = f'I heard you say: {prompt}'
    
    with st.chat_message('assistant'):
        st.markdown(res)
        
    add_to_session_history('assistant', res)

    

In [ ]:
!streamlit run language_app.py